<a href="https://colab.research.google.com/github/Liraken/P04-Weather-Risk/blob/main/LSTM_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikeras

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as test_train_split
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import make_scorer, r2_score
from google.colab import files
import tensorflow as tf
from scikeras.wrappers import KerasRegressor
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import ParameterGrid
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
uploaded = files.upload()

Saving stripped.csv to stripped (3).csv


In [3]:
class R2Callback(Callback):
    def __init__(self, X_train, y_train, patience=5, restore_best_weights=False,verbose=1):
        super(R2Callback, self).__init__()
        self.X_train = X_train
        self.y_train = y_train
        self.patience = patience
        self.best_r2 = -float('inf')  # Initialize best R^2 score
        self.verbose=verbose
        self.wait = 0
        self.restore_best_weights = restore_best_weights
        self.best_weights = None

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.X_train)
        r2 = r2_score(self.y_train, y_pred)
        if self.verbose>0:
          print(f"Epoch {epoch+1}, R^2: {r2}")

        # Check if current R^2 score is greater than the best R^2 score
        if r2 > self.best_r2:
            self.best_r2 = r2
            self.wait = 0
            if self.restore_best_weights:
                self.best_weights = self.model.get_weights()  # Save the best weights
        else:
            self.wait += 1  # Increment the counter

            # Check if we have reached the patience limit
            if self.wait >= self.patience:
                print(f"Stopping training as R^2 score hasn't improved for {self.patience} epochs.")
                if self.restore_best_weights:
                    print("Restoring best weights...")
                    self.model.set_weights(self.best_weights)  # Restore the best weights
                self.model.stop_training = True

    def reset(self):
        self.best_r2 = -float('inf')
        self.wait = 0
        self.best_weights = None

In [4]:
# Custom R^2 scorer
def r2_scorer(y_true, y_pred):
    return r2_score(y_true, y_pred)

# Define LSTM model
def create_model(n_layers=1, neurons=128, activation='relu', return_sequences=True,):
    model = Sequential()
    for _ in range(n_layers):
        model.add(LSTM(neurons, activation='relu', input_shape=(timesteps, features)))
        if return_sequences:
            model.add(LSTM(units=units//2, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [5]:
class CustomEarlyStopping(Callback):
    def __init__(self, monitor='val_loss', value=0.0163, verbose=0, patience=0):
        super(CustomEarlyStopping, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose
        self.patience = patience
        self.wait=0

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is None:
            raise ValueError("The monitored metric '{}' is not available.".format(self.monitor))

        if current >= self.value:
            self.wait += 1
            if self.wait>= self.patience:
              if self.verbose > 0:
                print(f"\nValidation loss reached {self.value}, stopping training.")
              self.model.stop_training = True
        else:
            self.wait = 0

In [6]:
def calculate_r_squared(y_true, y_pred):
    """
    Calculate the R^2 score.

    Parameters:
        y_true (array-like): The true values.
        y_pred (array-like): The predicted values.

    Returns:
        float: R^2 score.
    """
    y_true_mean = np.mean(y_true)
    ss_tot = np.sum((y_true - y_true_mean) ** 2)
    ss_res = np.sum((y_true - y_pred) ** 2)
    r2 = 1 - (ss_res / ss_tot)
    return r2

In [7]:
def create_sequences(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X.iloc[i:(i + time_steps)].values)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [8]:
df=pd.read_csv('stripped.csv')
df.head()

,BEGIN_DATE_TIME,DAMAGE_PROPERTY
0,29-OCT-96 17:10:00,0.0
1,21-OCT-96 09:00:00,0.0
2,20-OCT-96 19:15:00,0.0
3,23-JAN-96 04:00:00,0.0
4,17-JAN-96 09:00:00,0.0


In [9]:
# df=df.drop(columns=['Unnamed: 0','BEGIN_LAT','BEGIN_LON','END_LAT','END_LON', 'DURATION_SEC'])
# df.sort_values(by=['BEGIN_DATE_TIME'],inplace=True)
# df.head()
df=df[['BEGIN_DATE_TIME','DAMAGE_PROPERTY']]
df.sort_values(by=['BEGIN_DATE_TIME'],inplace=True)
df.head()


,BEGIN_DATE_TIME,DAMAGE_PROPERTY
200296,01-APR-00 00:00:00,0.0
202664,01-APR-00 00:00:00,0.0
202665,01-APR-00 00:00:00,0.0
202678,01-APR-00 00:00:00,0.0
202679,01-APR-00 00:00:00,0.0


In [10]:
len(df)

1635110

In [11]:
agg_functions={
    'DAMAGE_PROPERTY':'sum',
    'EVENT_COUNT':'first'
}

In [12]:
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].astype('datetime64[ns]')
df.sort_values(by=['BEGIN_DATE_TIME'],inplace=True)
df.head()

,BEGIN_DATE_TIME,DAMAGE_PROPERTY
4841,1996-01-01,0.0
3278,1996-01-01,0.0
450,1996-01-01,0.0
2396,1996-01-01,0.0
5373,1996-01-01,0.0


In [13]:
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].astype('datetime64[ns]')
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].dt.to_period('M')
df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].astype('datetime64[ns]')
df['EVENT_COUNT']=df.groupby('BEGIN_DATE_TIME')['BEGIN_DATE_TIME'].transform('count')
df=df.groupby('BEGIN_DATE_TIME').agg(agg_functions).reset_index()
# df['BEGIN_DATE_TIME'] = df['BEGIN_DATE_TIME'].map(pd.Timestamp.timestamp)
# df['dt_sin']=np.sin(2*np.pi*df['BEGIN_DATE_TIME'])
# df['dt_cos']=np.cos(2*np.pi*df['BEGIN_DATE_TIME'])
# df=df.drop(columns=['BEGIN_DATE_TIME'])
df.head()

,BEGIN_DATE_TIME,DAMAGE_PROPERTY,EVENT_COUNT
0,1996-01-01,830475710.0,6369
1,1996-02-01,531568800.0,4119
2,1996-03-01,175668080.0,4072
3,1996-04-01,723753290.0,4401
4,1996-05-01,542637240.0,6355


In [14]:
# df['BEGIN_DATE_TIME']=df['BEGIN_DATE_TIME'].astype('datetime64[ns]')
df['MONTH']=df['BEGIN_DATE_TIME'].dt.month
df['YEAR']=df['BEGIN_DATE_TIME'].dt.year
# df['DAY_OF_WEEK']=df['BEGIN_DATE_TIME'].dt.dayofweek
# df['WEEK_OF_YEAR']=df['BEGIN_DATE_TIME'].dt.isocalendar().week
df=df.drop(columns=['BEGIN_DATE_TIME'])
df.head()

,DAMAGE_PROPERTY,EVENT_COUNT,MONTH,YEAR
0,830475710.0,6369,1,1996
1,531568800.0,4119,2,1996
2,175668080.0,4072,3,1996
3,723753290.0,4401,4,1996
4,542637240.0,6355,5,1996


In [15]:
X=df.drop(columns=['DAMAGE_PROPERTY'])
y=df[['DAMAGE_PROPERTY']]

In [16]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Normalize the target
scaler_y = MinMaxScaler(feature_range=(0, 1))
# y_normalized =pd.DataFrame(scaler_y.fit_transform(y.reshape(-1,1)), columns=y.columns)
y_reshaped=y.values.reshape(-1,1)
y_normalized =pd.DataFrame(scaler_y.fit_transform(y_reshaped), columns=y.columns)

In [17]:
train_size = int(len(X_normalized) * 0.8)
test_size = len(X_normalized) - train_size


In [18]:
time_steps = 10
X_train, y_train = create_sequences(X_normalized.iloc[0:train_size], y_normalized.iloc[0:train_size], time_steps)
X_test, y_test = create_sequences(X_normalized.iloc[train_size:len(X_normalized)], y_normalized.iloc[train_size:len(X_normalized)], time_steps)

In [20]:
model = Sequential()
model.add(LSTM(256,return_sequences=True))
tf.keras.layers.Dropout(0.2)
model.add(LSTM(256,return_sequences=True))
tf.keras.layers.Dropout(0.2)
model.add(LSTM(256))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [24]:
model.fit(X_train, y_train, epochs=100000, verbose=0,callbacks=[r2_callback])

2/2 [==============================] - 0s 14ms/step
Stopping training as R^2 score hasn't improved for 15 epochs.
Restoring best weights...


In [21]:
def create_lstm_model(n_layers, n_neurons, n_timesteps, n_features):
    model = Sequential()
    for i in range(n_layers - 1):
      if i>0:
        model.add(LSTM(n_neurons,return_sequences=True))
        tf.keras.layers.Dropout(0.2)  # For single layer, keep return_sequences=True by default
    model.add(LSTM(n_neurons))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [22]:
param_grid = {
    'n_layers': [2,3, 4, 5,6],
    'n_neurons': [128, 256, 512,1024,2048],
    'time_steps': [5,7,10]
}

In [23]:
best_model = None
best_r2 = float(0)
r2_callback = R2Callback(X_test, y_test, patience=15, restore_best_weights=True, verbose=0)

In [26]:

for params in ParameterGrid(param_grid):
    print("Testing parameters:", params)
    X_train, y_train = create_sequences(X_normalized.iloc[0:train_size], y_normalized.iloc[0:train_size], params['time_steps'])
    X_test, y_test = create_sequences(X_normalized.iloc[train_size:len(X_normalized)], y_normalized.iloc[train_size:len(X_normalized)], params['time_steps'])
    model = create_lstm_model(params['n_layers'], params['n_neurons'], params['time_steps'], len(X_normalized.columns))
    r2_callback.reset()
    model.fit(X_train, y_train, epochs=100000, verbose=0,callbacks=[r2_callback])
    r_squared=calculate_r_squared(y_test, model.predict(X_test))
    print("R^2 value:",r_squared)
    if r_squared > best_r2:
        best_r2 = r_squared
        best_model = model

print(f"Best model parameters:{n_layers},{n_neurons},{n_timesteps}")
print(best_model.summary())

Streaming output truncated to the last 5000 lines.
Testing parameters: {'n_layers': 2, 'n_neurons': 2048, 'time_steps': 3}
2/2 [==============================] - 0s 39ms/step
Stopping training as R^2 score hasn't improved for 15 epochs.
Restoring best weights...
2/2 [==============================] - 0s 26ms/step
R^2 value: 0.02462459000902839
Testing parameters: {'n_layers': 2, 'n_neurons': 2048, 'time_steps': 4}
2/2 [==============================] - 0s 38ms/step
Stopping training as R^2 score hasn't improved for 15 epochs.
Restoring best weights...
2/2 [==============================] - 0s 38ms/step
R^2 value: 0.08298923040063177
Testing parameters: {'n_layers': 2, 'n_neurons': 2048, 'time_steps': 5}
2/2 [==============================] - 0s 39ms/step
Stopping training as R^2 score hasn't improved for 15 epochs.
Restoring best weights...
2/2 [==============================] - 0s 39ms/step
R^2 value: 0.05458127243364985
Testing parameters: {'n_layers': 2, 'n_neurons': 2048, 'time_ste

KeyboardInterrupt: 

In [29]:
best_model.summary()
print(best_r2)

Model: "sequential_171"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_273 (LSTM)             (None, None, 256)         266240    
                                                                 
 lstm_274 (LSTM)             (None, None, 256)         525312    
                                                                 
 lstm_275 (LSTM)             (None, 256)               525312    
                                                                 
 dense_171 (Dense)           (None, 1)                 257       
                                                                 
Total params: 1317121 (5.02 MB)
Trainable params: 1317121 (5.02 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
0.2533874860703017
